In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#DATA


In [2]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

def scale(image, label):
    # we make sure the value is a float
    image = tf.cast(image, tf.float32)
    # since the possible values for the inputs are 0 to 255 (256 different shades of grey)
    # if we divide each element by 255, we would get the desired result -> all elements will be between 0 and 1 
    image /= 255.

    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)
num_test=mnist_info.splits['test'].num_examples

BUFFER_SIZE=10000

shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test)

validation_inputs, validation_targets = next(iter(validation_data))

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
train_data

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [4]:
validation_data

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

#MODEL

In [5]:
input_size=784
outpusize=10
hidden_layer_size=100

model=tf.keras.Sequential([
                           tf.keras.layers.Flatten(input_shape=(28,28,1)),
                           tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                           tf.keras.layers.Dense(hidden_layer_size,activation='relu'), 
                           tf.keras.layers.Dense(outpusize,activation='softmax')                      
                          ])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#TRAINING

In [ ]:
NUM_EPOCHS=10
model.fit(train_data, epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/10
540/540 - 8s - loss: 0.3398 - accuracy: 0.9027 - val_loss: 0.1898 - val_accuracy: 0.9457
Epoch 2/10
540/540 - 4s - loss: 0.1411 - accuracy: 0.9580 - val_loss: 0.1152 - val_accuracy: 0.9650
Epoch 3/10
540/540 - 4s - loss: 0.0983 - accuracy: 0.9701 - val_loss: 0.0868 - val_accuracy: 0.9725
Epoch 4/10
540/540 - 4s - loss: 0.0779 - accuracy: 0.9761 - val_loss: 0.0816 - val_accuracy: 0.9747
Epoch 5/10
540/540 - 4s - loss: 0.0611 - accuracy: 0.9812 - val_loss: 0.0701 - val_accuracy: 0.9778
Epoch 6/10
540/540 - 4s - loss: 0.0513 - accuracy: 0.9842 - val_loss: 0.0642 - val_accuracy: 0.9805
Epoch 7/10
540/540 - 4s - loss: 0.0421 - accuracy: 0.9874 - val_loss: 0.0446 - val_accuracy: 0.9865
Epoch 8/10
540/540 - 4s - loss: 0.0361 - accuracy: 0.9886 - val_loss: 0.0430 - val_accuracy: 0.9872
Epoch 9/10
540/540 - 4s - loss: 0.0304 - accuracy: 0.9900 - val_loss: 0.0487 - val_accuracy: 0.9842
Epoch 10/10
540/540 - 4s - loss: 0.0254 - accuracy: 0.9927 - val_loss: 0.0366 - val_accuracy: 0.9892

#TEST

In [ ]:
test_loss,test_accuracy=model.evaluate(test_data)
print('Loss function={} and accuracy={}'.format(round(test_loss,3),round(test_accuracy,3)))

1/1 [==============================] - 0s 1ms/step - loss: 0.1210 - accuracy: 0.9782
Loss function=0.121 and accuracy=0.978
